In [1]:
import pandas as pd

In [11]:
dfTotal_external = pd.read_csv('../../Datasets/External_Validation_Datasets/External_Validation2.csv')


In [ ]:
dfTotal_external

,title,label
0,Origins: Hunting the Source of Covid-19,1
1,Totally Under Control: Trump and Covid-19,1
2,Newsday: Mass Covid-19 testing in Beijing,1
3,Covid-19 in the UK,1
4,Coronavirus: More than 100 TfL workers died fr...,1
...,...,...
30625,After COVID-19 outbreak in europe Muslims and...,0
30626,Saddam Hussein predicted the coronavirus outb...,0
30627,Eight COVID-19 patients in Ghana have recovered.,0
30628,People with type A blood are more prone to ge...,0


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
 

In [ ]:
model_path = '../../ML_Models/models/'

In [ ]:
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Validate on BERT

## Trained on FNN&CoAID

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

import pandas as pd
from datasets import Dataset
import numpy as np
from datasets import load_metric
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer("Attention is all you need")

def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
import numpy as np
from datasets import load_metric
from transformers import Trainer
from transformers import TrainingArguments

#Load saved model
import torch
model.load_state_dict(torch.load(model_path+'finetune_bert_on_CoAID_FNN.pt'))

#Dataset
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

#Eval Parameters
metric = load_metric("accuracy")
test_args = TrainingArguments("test_trainer", num_train_epochs=3)

#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

  0%|          | 0/31 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30630
  Batch size = 8


{'eval_accuracy': 0.8674502122102514,
 'eval_f1': 0.8734650626441438,
 'eval_loss': 0.5516829490661621,
 'eval_precision': 0.883877885707077,
 'eval_recall': 0.8632947264662395,
 'eval_runtime': 570.4063,
 'eval_samples_per_second': 53.699,
 'eval_steps_per_second': 6.713}

## Out of box

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_accuracy': 0.47006203068886715,
 'eval_f1': 0.0,
 'eval_loss': 0.7524212002754211,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 567.5186,
 'eval_samples_per_second': 53.972,
 'eval_steps_per_second': 6.747}

## finetune_bert_on_CoAID&GossipCop_only

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_CoAID&GossipCop_only.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.9047012732615083,
 'eval_f1': 0.9109599487539274,
 'eval_loss': 0.4938521087169647,
 'eval_precision': 0.9021811370914145,
 'eval_recall': 0.9199112863479546,
 'eval_runtime': 566.8569,
 'eval_samples_per_second': 54.035,
 'eval_steps_per_second': 6.755}

## CoAID 

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_CoAID.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.9455762324518446,
 'eval_f1': 0.9498088098033902,
 'eval_loss': 0.336826890707016,
 'eval_precision': 0.9288616689240916,
 'eval_recall': 0.9717225234105471,
 'eval_runtime': 568.4018,
 'eval_samples_per_second': 53.888,
 'eval_steps_per_second': 6.736}

## finetune_bert_on_CoAID&PolitiFact_only

In [ ]:
import torch
import numpy as np
from datasets import load_metric
from transformers import Trainer
from transformers import TrainingArguments


#Eval Parameters
metric = load_metric("accuracy")
test_args = TrainingArguments("test_trainer", num_train_epochs=3)

#Dataset
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

Parameter 'function'=<function tokenize_data at 0x7f9faa148200> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/31 [00:00<?, ?ba/s]

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_CoAID&PolitiFact_only.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

{'eval_accuracy': 0.9345086516487104,
 'eval_f1': 0.9375622509960159,
 'eval_loss': 0.35977908968925476,
 'eval_precision': 0.9474710619023654,
 'eval_recall': 0.92785855101035,
 'eval_runtime': 570.2881,
 'eval_samples_per_second': 53.71,
 'eval_steps_per_second': 6.714}

## finetune_bert_on_all_of_FNN.pt

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_all_of_FNN.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.5354554358472087,
 'eval_f1': 0.5653010723123453,
 'eval_loss': 1.5617187023162842,
 'eval_precision': 0.5606932913156778,
 'eval_recall': 0.5699852143913258,
 'eval_runtime': 569.7573,
 'eval_samples_per_second': 53.76,
 'eval_steps_per_second': 6.72}

## finetune_bert_on_GossipCop_only.pt

In [ ]:
import torch
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_GossipCop_only.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.40819458047665685,
 'eval_f1': 0.4827507490369526,
 'eval_loss': 1.9215482473373413,
 'eval_precision': 0.4496358900760113,
 'eval_recall': 0.5211310990635781,
 'eval_runtime': 568.2586,
 'eval_samples_per_second': 53.902,
 'eval_steps_per_second': 6.738}

##finetune_bert_on_PolitiFact_only

In [ ]:

import torch
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_PolitiFact_only.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.841005550114267,
 'eval_f1': 0.8362584896778966,
 'eval_loss': 0.6823464035987854,
 'eval_precision': 0.9205033308660252,
 'eval_recall': 0.7661409561360276,
 'eval_runtime': 568.4556,
 'eval_samples_per_second': 53.883,
 'eval_steps_per_second': 6.736}

# Roberta-Fake-News

## 1.Out-of-box

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

In [ ]:
#Dataset
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

#Eval Parameters
metric = load_metric("accuracy")
test_args = TrainingArguments("test_trainer", num_train_epochs=3)

#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

  0%|          | 0/31 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30630
  Batch size = 8


{'eval_accuracy': 0.39268690825987596,
 'eval_f1': 0.5589434749620638,
 'eval_loss': 1.2812670469284058,
 'eval_precision': 0.45432469935245146,
 'eval_recall': 0.7261582060128142,
 'eval_runtime': 561.9995,
 'eval_samples_per_second': 54.502,
 'eval_steps_per_second': 6.813}

## 2.Finetuned on CoAID 

In [ ]:
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

  0%|          | 0/31 [00:00<?, ?ba/s]

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_with_shuffle.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()


loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.8097616715638263,
 'eval_f1': 0.797582241984229,
 'eval_loss': 1.8956300020217896,
 'eval_precision': 0.9143767423337316,
 'eval_recall': 0.7072449482503697,
 'eval_runtime': 561.2516,
 'eval_samples_per_second': 54.574,
 'eval_steps_per_second': 6.822}

## 3.Fine-tuned on FNN

In [ ]:
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

  0%|          | 0/31 [00:00<?, ?ba/s]

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_FNN.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.4691478942213516,
 'eval_f1': 0.5309795777085498,
 'eval_loss': 1.4987866878509521,
 'eval_precision': 0.4992406161857236,
 'eval_recall': 0.567028092656481,
 'eval_runtime': 566.0035,
 'eval_samples_per_second': 54.116,
 'eval_steps_per_second': 6.765}

## 4.Fine-tuned on CoAID&FNN

In [ ]:
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

  0%|          | 0/31 [00:00<?, ?ba/s]

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_CoAID&FNN.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.7498857329415606,
 'eval_f1': 0.7313909049472319,
 'eval_loss': 0.9987914562225342,
 'eval_precision': 0.8487265033770038,
 'eval_recall': 0.6425579103006407,
 'eval_runtime': 562.4497,
 'eval_samples_per_second': 54.458,
 'eval_steps_per_second': 6.808}

## 5.Fine-tuned on CoAID&PolitiFact

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_CoAID&&PolitiFact_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.8429970617042115,
 'eval_f1': 0.8398654723452433,
 'eval_loss': 0.7105494737625122,
 'eval_precision': 0.9139068048409305,
 'eval_recall': 0.7769221291276491,
 'eval_runtime': 560.6595,
 'eval_samples_per_second': 54.632,
 'eval_steps_per_second': 6.829}

## 6.Fine-tuned on CoAID & GossipCop

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_CoAID&GossipCop_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.7403525954946131,
 'eval_f1': 0.7214458337711463,
 'eval_loss': 1.1903114318847656,
 'eval_precision': 0.8360256514327462,
 'eval_recall': 0.6344874322326269,
 'eval_runtime': 559.6243,
 'eval_samples_per_second': 54.733,
 'eval_steps_per_second': 6.842}

## 7.Fine-tuned on GossipCop

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_GossipCop_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.3505060398302318,
 'eval_f1': 0.5157961349364748,
 'eval_loss': 1.1367509365081787,
 'eval_precision': 0.42632976583246157,
 'eval_recall': 0.6527846229669788,
 'eval_runtime': 559.2367,
 'eval_samples_per_second': 54.771,
 'eval_steps_per_second': 6.847}

## 8.Fine-tuned on PolitiFact

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_PolitiFact_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.7453476983349657,
 'eval_f1': 0.7797978657331602,
 'eval_loss': 1.0394972562789917,
 'eval_precision': 0.7196977592496092,
 'eval_recall': 0.8508501724987678,
 'eval_runtime': 559.533,
 'eval_samples_per_second': 54.742,
 'eval_steps_per_second': 6.843}

# Fake-News-Bert-Detect

## 1. Out-of-box

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')

https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpgk1glzob


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/1bb953ac793c5ae15e9b5a3e1748d62cad5e9e5557408e989fb5d25f00ee6d0c.d6a300aa289d15142bf7dbad3b3a42b7c1ee1e0c874081ff0108003a2e307a2a
creating metadata file for /root/.cache/huggingface/transformers/1bb953ac793c5ae15e9b5a3e1748d62cad5e9e5557408e989fb5d25f00ee6d0c.d6a300aa289d15142bf7dbad3b3a42b7c1ee1e0c874081ff0108003a2e307a2a
https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/vocab.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpks4p7k7j


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/5a3b85c1b5bf39e7feabb076838b178aa37c4e860c428b2fbd40ef1972f679a0.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
creating metadata file for /root/.cache/huggingface/transformers/5a3b85c1b5bf39e7feabb076838b178aa37c4e860c428b2fbd40ef1972f679a0.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpv2zfc1m4


...

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/6a4467084bef307312ff1896f1584b5a7f28fa88d2c2c39fa2e4905e6e2bdd2c.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0
creating metadata file for /root/.cache/huggingface/transformers/6a4467084bef307312ff1896f1584b5a7f28fa88d2c2c39fa2e4905e6e2bdd2c.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0
loading file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/5a3b85c1b5bf39e7feabb076838b178aa37c4e860c428b2fbd40ef1972f679a0.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
loading file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/abccee4c0ea0661b34063ece912c47e6d578f8ab52d66c02ea3387a3b5eb79b9.5d12962c5ee615a4c803841266e9c3be9a691a9

Downloading:   0%|          | 0.00/735 [00:00<?, ?B/s]

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
creating metadata file for /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/6cef7d197f6d86a75be415374a6577454a31076c29da2583b8a441b6f273d6b0.c3091640aebed39d220ea7006739d06210601f14925e4fd5acfc493711b8081d
creating metadata file for /root/.cache/huggingface/transformers/6cef7d197f6d86a75be415374a6577454a31076c29da2583b8a441b6f273d6b0.c3091640aebed39d220ea7006739d06210601f14925e4fd5acfc493711b8081d
loading weights file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/6cef7d197f6d86a75be415374a6577454a31076c29da2583b8a441b6f273d6b0.c3091640aebed39d220ea7006739d06210601f14925e4fd5acfc493711b8081d
All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at jy46604790/Fake-News-Bert-Detect.
If your task is 

In [ ]:
#Dataset
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

#Eval Parameters
metric = load_metric("accuracy")
test_args = TrainingArguments("test_trainer", num_train_epochs=3)

#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

  0%|          | 0/31 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30630
  Batch size = 8


{'eval_accuracy': 0.4899771465883121,
 'eval_f1': 0.15437912742232326,
 'eval_loss': 2.6161410808563232,
 'eval_precision': 0.6360392506690455,
 'eval_recall': 0.08785115820601282,
 'eval_runtime': 560.6432,
 'eval_samples_per_second': 54.634,
 'eval_steps_per_second': 6.83}

## 2.Finetuned on CoAID

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_shuffle_train_on_all_CoAID.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.9418543911198172,
 'eval_f1': 0.9461396558502434,
 'eval_loss': 0.36190667748451233,
 'eval_precision': 0.9291951291951291,
 'eval_recall': 0.9637136520453425,
 'eval_runtime': 559.3718,
 'eval_samples_per_second': 54.758,
 'eval_steps_per_second': 6.845}

## 3.Fine-tuned on FNN 

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_Finetune_on_FNN.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.5299379693111329,
 'eval_f1': 0.692757458068371,
 'eval_loss': 0.6928958892822266,
 'eval_precision': 0.5299379693111329,
 'eval_recall': 1.0,
 'eval_runtime': 558.3641,
 'eval_samples_per_second': 54.857,
 'eval_steps_per_second': 6.858}

## 4.Fine-tuned on CoAID&FNN 

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_Finetune_on_CoAID&FNN.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.778256611165524,
 'eval_f1': 0.7691366417403126,
 'eval_loss': 1.1686644554138184,
 'eval_precision': 0.8579011222323324,
 'eval_recall': 0.6970182355840315,
 'eval_runtime': 559.1448,
 'eval_samples_per_second': 54.78,
 'eval_steps_per_second': 6.848}

## 5.Fine-tuned on CoAID&PolitiFact

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_finetuned_on_CoAID&PolitiFact_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.9163238654913484,
 'eval_f1': 0.9205985315530221,
 'eval_loss': 0.46558281779289246,
 'eval_precision': 0.9259051536112669,
 'eval_recall': 0.915352390340069,
 'eval_runtime': 558.0481,
 'eval_samples_per_second': 54.888,
 'eval_steps_per_second': 6.861}

## 6.Fine-tuned on CoAID & GossipCop 

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_finetuned_on_CoAID&GossipCop_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.7271955599085863,
 'eval_f1': 0.7157436385902842,
 'eval_loss': 0.8935675621032715,
 'eval_precision': 0.799149194773625,
 'eval_recall': 0.6481025135534746,
 'eval_runtime': 558.1648,
 'eval_samples_per_second': 54.876,
 'eval_steps_per_second': 6.86}

## 7.Fine-tuned on GossipCop 

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_finetuned_on_GossipCop_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.37078028077048647,
 'eval_f1': 0.5148150945296176,
 'eval_loss': 1.2745856046676636,
 'eval_precision': 0.4352730833085011,
 'eval_recall': 0.6299285362247412,
 'eval_runtime': 557.8817,
 'eval_samples_per_second': 54.904,
 'eval_steps_per_second': 6.863}

## 8.Fine-tuned on PolitiFact

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_finetuned_on_PolitiFact_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.7633692458374143,
 'eval_f1': 0.7721041378442963,
 'eval_loss': 1.1192243099212646,
 'eval_precision': 0.7884664782943746,
 'eval_recall': 0.7564070970921636,
 'eval_runtime': 558.7716,
 'eval_samples_per_second': 54.817,
 'eval_steps_per_second': 6.853}

#XLNet

## Import XLNet tokenizer, used to convert our text into tokens that correspond to XLNet’s vocabulary.

In [ ]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import io
import numpy as np
import matplotlib.pyplot as plt

from unicodedata import name
import pandas as pd
import glob, os
from datasets import Dataset
import numpy as np
from datasets import load_metric

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install sentencepiece 
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 7.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 8.7 MB/s 
     |████████████████████████████████| 1.8 MB 53.9 MB/s 
     |████████████████████████████████| 9.1 MB 49.6 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 181 kB 56.5 MB/s 
     |████████████████████████████████| 145 kB 70.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 4.3 MB 57.5 MB/s 
     |████████████████████████████████| 164 kB 68.9 MB/s 
     |████████████████████████████████| 111 kB 77.4 MB/s 
     |████████████████████████████████| 78 kB 8.9 MB/s 
     |████████████████████████████████| 232 kB 74.5 MB/s 
     |████████████████████████████████| 133 kB 72.5 MB/s

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)


Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

## 1. Out-of-box 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
{
        'accuracy': accuracy_score(flat_true_labels, flat_predictions),
        'f1': f1_score(flat_true_labels, flat_predictions),
        'precision': precision_score(flat_true_labels, flat_predictions),
        'recall': recall_score(flat_true_labels, flat_predictions)
    }

{'accuracy': 0.5300032647730982,
 'f1': 0.6927870251813915,
 'precision': 0.5299725741151887,
 'recall': 1.0}

## 2.Finetuned on CoAID 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_CoAID.pt'))

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

<All keys matched successfully>

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set
model.cuda()
# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
{
        'accuracy': accuracy_score(flat_true_labels, flat_predictions),
        'f1': f1_score(flat_true_labels, flat_predictions),
        'precision': precision_score(flat_true_labels, flat_predictions),
        'recall': recall_score(flat_true_labels, flat_predictions)
    }

{'accuracy': 0.9116552399608228,
 'f1': 0.9223440280089535,
 'precision': 0.8633286773396368,
 'recall': 0.990019714144899}

## 3.Fine-tuned on FNN 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_FNN.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
{
        'accuracy': accuracy_score(flat_true_labels, flat_predictions),
        'f1': f1_score(flat_true_labels, flat_predictions),
        'precision': precision_score(flat_true_labels, flat_predictions),
        'recall': recall_score(flat_true_labels, flat_predictions)
    }

{'accuracy': 0.6650669278485145,
 'f1': 0.7489170072689003,
 'precision': 0.6212693385308807,
 'recall': 0.9425825529817644}

## 4.Fine-tuned on CoAID&FNN

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_CoAID_FNN.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
{
        'accuracy': accuracy_score(flat_true_labels, flat_predictions),
        'f1': f1_score(flat_true_labels, flat_predictions),
        'precision': precision_score(flat_true_labels, flat_predictions),
        'recall': recall_score(flat_true_labels, flat_predictions)
    }

{'accuracy': 0.9206333659810643,
 'f1': 0.9259631490787269,
 'precision': 0.9156176594591339,
 'recall': 0.9365450961064564}

## 5.Fine-tuned on CoAID&PolitiFact 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_CoAID_Politi.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
{
        'accuracy': accuracy_score(flat_true_labels, flat_predictions),
        'f1': f1_score(flat_true_labels, flat_predictions),
        'precision': precision_score(flat_true_labels, flat_predictions),
        'recall': recall_score(flat_true_labels, flat_predictions)
    }

{'accuracy': 0.8985308521057787,
 'f1': 0.9011953204476094,
 'precision': 0.9310299527062533,
 'recall': 0.8732134056185313}

## 6.Fine-tuned on CoAID & GossipCop 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_CoAID_Gossip.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
{
        'accuracy': accuracy_score(flat_true_labels, flat_predictions),
        'f1': f1_score(flat_true_labels, flat_predictions),
        'precision': precision_score(flat_true_labels, flat_predictions),
        'recall': recall_score(flat_true_labels, flat_predictions)
    }

{'accuracy': 0.8668625530525629,
 'f1': 0.8776477647764777,
 'precision': 0.8554216867469879,
 'recall': 0.9010596352883193}

## 7.Fine-tuned on GossipCop

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_Gossip.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
{
        'accuracy': accuracy_score(flat_true_labels, flat_predictions),
        'f1': f1_score(flat_true_labels, flat_predictions),
        'precision': precision_score(flat_true_labels, flat_predictions),
        'recall': recall_score(flat_true_labels, flat_predictions)
    }

{'accuracy': 0.493111328762651,
 'f1': 0.5680743337228064,
 'precision': 0.517906056609516,
 'recall': 0.6290044356826022}

## 8.Fine-tuned on PolitiFact

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_Politi.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
{
        'accuracy': accuracy_score(flat_true_labels, flat_predictions),
        'f1': f1_score(flat_true_labels, flat_predictions),
        'precision': precision_score(flat_true_labels, flat_predictions),
        'recall': recall_score(flat_true_labels, flat_predictions)
    }

{'accuracy': 0.7942213516160627,
 'f1': 0.81664,
 'precision': 0.7736317036873726,
 'recall': 0.8647116806308527}